# Assignment :

# Part 1 - Prepare data

# 1. Code to scrape the Wikipedia page - List of postal codes of Canada: M

In [17]:
import pandas as pd

In [18]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [19]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# 2. Ignore cells with a borough that is Not assigned. 


In [20]:
dfDropNA = df[df.Borough != 'Not assigned'].reset_index(drop=True)   # we create new df for convenience and testing instead of overwriting using inplace=True

In [23]:
dfDropNA.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# 3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


In [52]:
# use Pandas GroupBy and use apply() to invoke Join on the data to concatenante the Neighbourhood
dfGroupByPCB = dfDropNA.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
dfGroupByPCB.head()   # we dont see any duplicate postal codes with different neighbourhood

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [41]:
# Get Neighbourhoods
dfGroupByPCBN = dfGroupByPCB['Neighbourhood'].apply(', '.join).reset_index()
dfGroupByPCBN.head()


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# 4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [112]:
# Replace Not assigned neighbourhood with Borough 
dfGroupByPCBN.Neighbourhood.fillna(dfGroupByPCBN.Borough,inplace=True)
dfGroupByPCBN.head()


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# 5. Use the .shape method to print the number of rows of your dataframe.

In [113]:
dfGroupByPCBN.shape

(103, 3)

# 6. We need to get the latitude and the longitude coordinates of each neighborhood.



In [114]:
dfLatLon = pd.read_csv('https://cocl.us/Geospatial_data')
dfLatLon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# 7. Create a new dataframe with both data

In [152]:
dfLOC = pd.merge(dfGroupByPCBN,dfLatLon,on=dfLatLon['Postal Code'])
dfLOC.head()

,key_0,Postal Code_x,Borough,Neighbourhood,Postal Code_y,Latitude,Longitude
0,M1B,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [153]:
# for convenience we will retain only one Postal Code column - clean up
dfLOC.drop(columns=['key_0'],inplace=True)
dfLOC.drop(columns=['Postal Code_y'],inplace=True)
dfLOC.rename(columns={"Postal Code_x":"Postal Code"},inplace=True)
dfLOC.head()



,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 2 . Clustering and the plotting of the neighbourhoods  - toronto



In [161]:
# we are interested only in toronto neighbourhood
dfToronto = dfLOC[dfLOC.Borough.str.contains('Toronto')]
dfToronto

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


# Visualizing using map

In [155]:
# pre-requisites

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')

/bin/bash: conda: command not found
Folium installed


In [162]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(dfToronto['Latitude'],dfToronto['Longitude'],dfToronto['Borough'],dfToronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

# Clustering the neighbourhoods - using Kmeans



In [163]:
# use only numeric columns to prepare labels
k=5
dfToronto_clustering = dfToronto.drop(['Postal Code','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(dfToronto_clustering)
print(kmeans.labels_)
dfToronto.insert(0, 'Cluster Labels', kmeans.labels_)
dfToronto.head()

[4 4 4 4 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 2 2 3 3 3 1 1 1 3 0 3 3 0 0 0
 1 4]


,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,4,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,4,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,4,M4M,East Toronto,Studio District,43.659526,-79.340923
44,2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


# Finally map the clusters

In [165]:
import numpy as np

# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['Neighbourhood'], dfToronto['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters